<p>Example script to extract 1 month of hourly MLST MSG product over a domain</p>

In [1]:
import datetime as dt
import thredds_lsasaf_utils as tlu
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

import rasterio
from rasterio.mask import mask
from shapely.geometry import Point
from shapely import wkt

import statsmodels.api as sm
from statsmodels.genmod.families import Gaussian
from statsmodels.genmod.families.links import Power
from statsmodels.genmod.families import Gamma
from statsmodels.genmod.families.links import log, identity

import pickle


In [2]:
def download_data(dstart, dend, product_freq, LatLonBox):

    # Change here your user credentials
    server_user = "karpagam"
    server_passwd = "chip-chop-2025"

    # Change here the product details
    # Go to https://thredds.lsasvcs.ipma.pt/thredds/catalog/catalog.html
    # Navigate selecting satelite, product, format, and data to find the product_path and product file name
    # This is an example for the MSG MLST

    product_path = "/MSG/MLST/NETCDF/"
    product_fname = "NETCDF4_LSASAF_MSG_LST_MSG-Disk"
    NcvarsLoad = ['LST'] # list of netcdf variables to load from remote files

    # Initialize product details
    product = tlu.lsa_product(product_path,product_fname)
    product.user = server_user
    product.passwd = server_passwd

    # list of slots to be processed:
    slot_list = tlu.gen_slot_list(dstart, dend, product_freq)
    print(f"Will load:{len(slot_list)} files: {slot_list[0]} to {slot_list[-1]}")

    # Load data
    ds_full = tlu.load_product_slots_domain(product, slot_list, NcvarsLoad, LatLonBox=LatLonBox)

    # Extract the data array (assuming the variable name is 'temperature')
    data_array = ds_full['LST']

    # Step 1: Extract the temperature DataArray
    temperature_da = ds_full['LST']

    # Step 2: Stack dimensions (combine 'time', 'lat', and 'lon')
    stacked = temperature_da.stack(points=('time', 'lat', 'lon'))

    # Step 3: Reset the index and convert to DataFrame
    df = stacked.reset_index(['time', 'lat', 'lon']).to_dataframe(name='temperature').reset_index(drop=True)

    # Step 4: Add an 'hour' column 'day', 'month' and 'year'
    df['hour'] = df['time'].dt.hour
    df['day'] = df['time'].dt.day
    df['month'] = df['time'].dt.month
    df['year'] = df['time'].dt.year

    # Create geometry from latitude and longitude
    geometry = [Point(xy) for xy in zip(df['lon'], df['lat'])]

    # Convert to GeoDataFrame
    gdf = gpd.GeoDataFrame(df, geometry=geometry)

    # Set the Coordinate Reference System (CRS) - assuming WGS84 (EPSG:4326)
    gdf.set_crs(epsg=4326, inplace=True)

    return gdf

In [ ]:
# Change here your user credentials
# server_user = "karpagam"
# server_passwd = "chip-chop-2025"

# # Change here the product details
# # Go to https://thredds.lsasvcs.ipma.pt/thredds/catalog/catalog.html
# # Navigate selecting satelite, product, format, and data to find the product_path and product file name
# # This is an example for the MSG MLST
# product_path = "/MSG/MLST/NETCDF/"
# product_fname = "NETCDF4_LSASAF_MSG_LST_MSG-Disk"
# NcvarsLoad = ['LST'] # list of netcdf variables to load from remote files

# time period to process
dstart = dt.datetime(2024, 6, 1, 0, 0, 0) # start slot
dend = dt.datetime(2024, 8, 31, 23, 0, 0)   # end slot
product_freq = "h" # hourly frequency

# Define latitude/longitude domain to load [lat_min,lat_max,lon_min,lon_max,]
LatLonBox = [41.6899140207028722, 42.0902931428349447, 12.2299337725884012, 12.7300258912577391] # Rome

# gdf = download_data(dstart, dend, product_freq, LatLonBox)


In [3]:
def read_fused_data(filename):

    # Replace 'your_file.csv' with the path to your CSV file
    file_path = filename

    # Load the CSV file into a pandas DataFrame
    df = pd.read_csv(file_path)

    # Ensure the CSV contains 'lat' and 'lon' columns
    if 'lat' not in df.columns or 'lon' not in df.columns:
        raise ValueError("The CSV file must contain 'lat' and 'lon' columns")

    # Create a GeoDataFrame
    geometry = [Point(xy) for xy in zip(df['lon'], df['lat'])]
    gdf = gpd.GeoDataFrame(df, geometry=geometry)

    # Set the coordinate reference system (CRS) if known, e.g., WGS84 (EPSG:4326)
    gdf.set_crs(epsg=4326, inplace=True)

    return gdf

In [4]:
gdf = read_fused_data('fused_geo_data_june_to_august.csv')

In [ ]:
gdf.head()

In [ ]:

# Load the pickle file
# with open('landuse_profile.pkl', 'rb') as file:
#     loaded_geo_lulc_dict = pickle.load(file)

# print("Loaded landuse profile:", loaded_geo_lulc_dict)


In [ ]:
# Create GeoDataFrame from dictionary
# lulc_gdf = gpd.GeoDataFrame(
#     list(loaded_geo_lulc_dict.items()),
#     columns=['geometry', 'lulc_values'],
#     geometry='geometry'
# )
# lulc_gdf.set_crs(gdf.crs, inplace=True)


In [ ]:
# sample_point = gdf['geometry'].iloc[0]
# print("Sample GeoDataFrame Point:", sample_point)
# print("Is this point in the dictionary?", sample_point in loaded_geo_lulc_dict)


In [26]:
# # Round coordinates to a consistent precision (e.g., 6 decimal places)
# def round_geometry(geom, precision=6):
#     return Point(round(geom.x, precision), round(geom.y, precision))

# # Update GeoDataFrame geometry
# gdf['geometry'] = gdf['geometry'].apply(lambda geom: round_geometry(geom))

# # Update LULC dictionary keys
# loaded_geo_lulc_dict = {
#     round_geometry(key): value for key, value in loaded_geo_lulc_dict.items()
# }


In [ ]:
# # Map LULC values to the GeoDataFrame using the geometry column
# gdf['lulc_values'] = gdf['geometry'].map(loaded_geo_lulc_dict)

# # Display the updated GeoDataFrame
# print(gdf.head())


In [ ]:

# # Add the lulc columns from the pickle file to the geopandas dataframe, using the point geometry as the key
# # Ensure geometry column matches the keys in the dictionary
# # If the dictionary keys are WKT strings, convert them to geometry objects for comparison
# # loaded_geo_lulc_dict = {Point(wkt.loads(k)): v for k, v in loaded_geo_lulc_dict.items()}

# # Extract the LULC column names from the dictionary values (assuming uniform structure)
# lulc_columns = [f'lulc_{i+1}' for i in range(len(next(iter(loaded_geo_lulc_dict.values()))))]

# # Add LULC columns to GeoDataFrame
# for idx, geom in enumerate(gdf['geometry']):
#     lulc_values = loaded_geo_lulc_dict.get(geom, [None] * len(lulc_columns))
#     for col, value in zip(lulc_columns, lulc_values):
#         gdf.loc[idx, col] = value

# # Display the updated GeoDataFrame
# print(gdf.head())

In [ ]:
# # Convert geometry to WKT format
# gdf['geometry'] = gdf['geometry'].apply(lambda geom: geom.wkt)

# # Save to CSV
# output_csv_path = 'fused_geo_data_june_to_august.csv'
# gdf.to_csv(output_csv_path, index=False)

# print(f"GeoDataFrame saved to {output_csv_path}")


In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import patsy
import statsmodels.api as sm

# Define target and LULC columns
gdf = gdf.drop(columns=['lulc_values'])

target = 'temperature'  # Replace with your target column name
lulc_columns = [col for col in gdf.columns if col.startswith('lulc_')]  # LULC proportions
lulc_columns

['lulc_1', 'lulc_2', 'lulc_3', 'lulc_4', 'lulc_5', 'lulc_6', 'lulc_7']

In [7]:

# Convert 'month' to a categorical variable
gdf['month'] = pd.Categorical(gdf['month'])

# Convert `hour` to categorical
gdf['hour'] = pd.Categorical(gdf['hour'])

# Drop rows with missing values
gdf = gdf.dropna(subset=[target, 'hour', 'month'] + lulc_columns)

# Update the interaction formula to include 'month'
# interaction_formula = f"{target} ~ C(hour) * C(month) * ({' + '.join(lulc_columns)})"
interaction_formula = f"{target} ~ C(hour) * ({' + '.join(lulc_columns)}) + C(month)"

# Step 2: Split the data into training and testing sets (75-25 split)
train_data, test_data = train_test_split(gdf, test_size=0.75, random_state=42)

# Step 3: Generate design matrices for train and test sets
y_train, X_train = patsy.dmatrices(interaction_formula, data=train_data, return_type='dataframe')
y_test, X_test = patsy.dmatrices(interaction_formula, data=test_data, return_type='dataframe')

# Step 4: Fit the GLM model on the training data
gamma_model = sm.GLM(y_train, X_train, family=sm.families.Gamma(link=sm.families.links.identity()))
gamma_results = gamma_model.fit()

# Step 5: Display model summary
print(gamma_results.summary())


/home/emaximus/workspace/lsasaf_data_access/.venv/lib/python3.10/site-packages/statsmodels/genmod/families/links.py:13: FutureWarning: The identity link alias is deprecated. Use Identity instead. The identity link alias will be removed after the 0.15.0 release.
  warnings.warn(
/home/emaximus/workspace/lsasaf_data_access/.venv/lib/python3.10/site-packages/statsmodels/genmod/generalized_linear_model.py:308: DomainWarning: The identity link function does not respect the domain of the Gamma family.
  warnings.warn((f"The {type(family.link).__name__} link function "


: 

In [ ]:

# Make predictions on the test data
y_pred = gamma_results.predict(X_test)

# Plot observed vs predicted values
plt.figure(figsize=(8, 6))
plt.scatter(y_test, y_pred, alpha=0.7, edgecolors='k', label='Data points')
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], color='red', label='Perfect Fit')
plt.xlabel("Observed Temperature")
plt.ylabel("Predicted Temperature")
plt.title("Observed vs Predicted Temperature")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
import pickle

# Save the GLM model to a pickle file
with open('glm_model.pkl', 'wb') as file:
    pickle.dump(gamma_results, file)

print("GLM model saved as 'glm_model.pkl'")

# Save the model summary to a text file
with open('glm_model_summary.txt', 'w') as file:
    file.write(gamma_results.summary().as_text())

print("GLM model summary saved as 'glm_model_summary.txt'")


In [ ]:
import matplotlib.pyplot as plt

# Ensure the 'time' column is in datetime format
gdf['time'] = pd.to_datetime(gdf['time'])

# Sort the GeoDataFrame by time for proper plotting
gdf = gdf.sort_values('time')

# Plot temperature over time
plt.figure(figsize=(10, 6))
plt.plot(gdf['time'], gdf['temperature'], label='Temperature', color='blue')

# Customize the plot
plt.title('Temperature Time Series')
plt.xlabel('Time')
plt.ylabel('Temperature')
plt.grid(True)
plt.legend()
plt.tight_layout()

# Display the plot
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Ensure the 'time' column is in datetime format
gdf['time'] = pd.to_datetime(gdf['time'])

# Extract the month from the 'time' column if not already done
gdf['month'] = gdf['time'].dt.month

# Filter data for August
august_data = gdf[gdf['month'] == 6]

# Sort by time
august_data = august_data.sort_values('time')

# Plot temperature over time for August as points
plt.figure(figsize=(10, 6))
plt.scatter(august_data['time'], august_data['temperature'], label='Temperature (August)', color='orange', alpha=0.8)

# Customize the plot
plt.title('Temperature Time Series for August (Points)')
plt.xlabel('Time')
plt.ylabel('Temperature')
plt.grid(True)
plt.legend()
plt.tight_layout()

# Display the plot
plt.show()


In [ ]:
import numpy as np
from sklearn.metrics import mean_squared_log_error
from scipy.stats import pearsonr

# Compute RMSLE (Root Mean Squared Logarithmic Error)
rmsle = np.sqrt(mean_squared_log_error(y_test, y_pred))
print(f"RMSLE: {rmsle:.4f}")


In [ ]:
import matplotlib.pyplot as plt

# Create a combined 'month-hour' column for better visualization
test_data['month_hour'] = test_data['month'].astype(str) + '-' + test_data['hour'].astype(str)

# Add predictions to the test dataset
test_data['observed'] = y_test.values
test_data['predicted'] = y_pred

# Sort by the combined 'month-hour' index for proper plotting
test_data = test_data.sort_values(by=['month', 'hour'])

# Plot observed vs predicted as a time series
plt.figure(figsize=(14, 6))
plt.plot(test_data['month_hour'], test_data['observed'], label='Observed Temperature', marker='o', linestyle='-', color='blue')
plt.plot(test_data['month_hour'], test_data['predicted'], label='Predicted Temperature', marker='x', linestyle='--', color='red')
plt.xlabel('Month-Hour')
plt.ylabel('Temperature')
plt.title('Time Series: Observed vs Predicted Temperature')
plt.xticks(rotation=90)  # Rotate x-axis labels for better readability
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()
